<a href="https://colab.research.google.com/github/around-star/Speech-Recognition/blob/main/Speech_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
from itertools import groupby
import pandas as pd
import os
import sklearn
!pip install pydub
from pydub import AudioSegment
import time
!pip install jiwer
from jiwer import wer

     |████████████████████████████████| 51kB 4.5MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144791 sha256=40ad40ebd1fa48fa5f2fe4e6774f09433353a32e01b8d61c47c76817d86004e6
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein


## Subwords github repo clone

In [ ]:
!git clone https://github.com/bheinzerling/bpemb.git

Cloning into 'bpemb'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 385 (delta 25), reused 30 (delta 13), pack-reused 338
Receiving objects: 100% (385/385), 620.22 KiB | 1.85 MiB/s, done.
Resolving deltas: 100% (221/221), done.


In [ ]:
pip install sentencepiece

     |████████████████████████████████| 1.1MB 10.9MB/s 


In [ ]:
from bpemb.bpemb import BPEmb

In [ ]:
bpemb_en = BPEmb(lang='en', dim = 50)

100%|██████████| 1924908/1924908 [00:00<00:00, 23865354.19B/s]

downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs10000.d50.w2v.bin.tar.gz


In [ ]:
bpemb_en.words

## Kaggle Common voice dataset download 

In [ ]:
!apt install kaggle

os.environ['KAGGLE_USERNAME'] = "andr3w88" # username from the json file
os.environ['KAGGLE_KEY'] = "67e85b3d3f682d0ae9bda7a78dfa7df0" # key from the json file

!kaggle datasets download -d mozillaorg/common-voice # api copied from kaggle

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package kaggle
100% 12.0G/12.0G [05:06<00:00, 51.9MB/s]
100% 12.0G/12.0G [05:06<00:00, 42.2MB/s]


In [ ]:
!unzip -q common-voice.zip -d common-voice/

## Code

**UTILS**

In [ ]:
def init_vocab():

    alphabet = "abcdefghijklmnopqrstuvwxyz'"
    alphabets = ['', ' '] + [c for c in alphabet]

    return alphabets


def load_audio(path):
    """Loads Audio and returns the vector"""
    sound = AudioSegment.from_mp3(path)
    sound = tf.cast(sound.get_array_of_samples(), tf.float32)
    sampled_sound = [sound[x] for x in range(0, len(sound), 48)]
    return sampled_sound


def compute_mel_spec(audio_arr, sample_rate = 15200, n_mel_bins = 80, frame_length = 0.025, frame_stride = 0.001, hertz_low = 125.0, hertz_high = 7600.0):
    sample_rate_f = tf.cast(sample_rate, tf.float32)

    frame_length = tf.cast(tf.round(frame_length * sample_rate_f), tf.int32)
    frame_stride = tf.cast(tf.round(frame_stride * sample_rate_f), tf.int32)

    stfts = tf.signal.stft(audio_arr, frame_length = frame_length, frame_step = frame_stride)

    mag_specs = tf.abs(stfts)
    num_spec_bins = tf.shape(mag_specs)[-1]

    linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(num_mel_bins = n_mel_bins, num_spectrogram_bins = num_spec_bins,
                                                                        sample_rate = sample_rate_f, lower_edge_hertz = hertz_low,
                                                                        upper_edge_hertz = hertz_high)

    mel_specs = tf.tensordot(mag_specs, linear_to_mel_weight_matrix, 1)
    mel_specs.set_shape(tf.concat([tf.shape(mag_specs)[:-1], tf.shape(linear_to_mel_weight_matrix)[-1:]], 0))

    log_mel_specs = tf.math.log(mel_specs + 1e-6)
    log_mel_specs -= (tf.reduce_mean(log_mel_specs, axis=0) + 1e-8) # Zero centered
    #log_mel_specs = log_mel_specs / tf.math.reduce_std(log_mel_specs, axis = 0) # Standarization
    
    return log_mel_specs


def ctc_greedy_decode(y_pred):
    decoded = []

    for i in range(np.shape(y_pred)[0]):
        decoded_batch = []
        for j in range(np.shape(y_pred)[1]):
            c = np.argmax(y_pred[i][j])
            decoded_batch.append(c)

        temp = [k for k , g in groupby(decoded_batch)]
        temp = [x for x in temp if x!=0]
        
        decoded.append(temp)

    return decoded

def joint(model, f, g):
    dense_1 = model.layers[-2]
    dense_2 = model.layers[-1]

    joint = (tf.expand_dims(f, axis = 2) + tf.expand_dims(g, axis = 1))

    outputs = dense_1(joint)
    outputs = dense_2(outputs)

    return outputs[0][0][-1][:]

def rnnt_greedy_decode(model, input):
    decoded = []
    input = tf.expand_dims(input, axis=0)
    encoder = model.layers[2]
    prediction_network = model.layers[3]

    start_token = tf.constant([0])

    encoded = encoder(input)
    enc_length = np.shape(input)[0]
    output = np.expand_dims(start_token, axis = 0)
    outputs = np.expand_dims(start_token, axis = 0)

    time_cond = True
    i = 0
    while (time_cond):
        inp_enc = tf.expand_dims(encoded[:][i][:], axis = 1)

        dec_end = True
        while (dec_end):
            
            pred_out = prediction_network(output)
            
            preds = joint(model, inp_enc, pred_out)

            pred_id = tf.argmax(preds, axis=-1)
            
            if (pred_id == 0):
                dec_cond = False
            else :
                outputs = np.concatenate([outputs, [[pred_id]]], axis = 1)
                output = np.array([[pred_id]])
        i+=1
        if (i>=enc_length):
            time_cond = False

    return outputs      

**BATCH GENERATOR**

In [ ]:

class DataGen :
    def __init__(self):
        self.x_train = []
        self.y_train = []
        self.logit_length = []
        self.label_length = []
        

    def create_char_data(self, path, csv_path, vocab):
        train_csv = pd.read_csv(csv_path)
        for i in range(len(train_csv["text"])):
            if (i>=200):
                break
            filename = train_csv.iloc[i]["filename"]
            audio_vec = load_audio(os.path.join(path, filename))
            feature_vec = compute_mel_spec(audio_vec)
            self.x_train.append(feature_vec)
            label = train_csv.iloc[i]["text"]
            labels = [vocab.index(j) for j in label]
            self.y_train.append(labels)
            self.label_length.append(len(label))
            self.logit_length.append(np.array(feature_vec).shape[0])

        #self.x_train = self.pad_feature(self.x_train)
        #self.y_train = self.pad_label(self.y_train)

    def create_wordpiece_data(self, path, csv_path):
        train_csv = pd.read_csv(csv_path)
        labels = []
        for i in range(len(train_csv["text"])):
            if (i>=100):
                break
            filename = train_csv.iloc[i]["filename"]
            audio_vec = load_audio(os.path.join(path, filename))
            feature_vec = compute_mel_spec(audio_vec)
            self.x_train.append(feature_vec)
            label = train_csv.iloc[i]["text"]
            labels.append(label)
            self.logit_length.append(np.array(feature_vec).shape[0])

        tokenizer = tf.keras.preprocessing.text.Tokenizer()
        tokenizer.fit_on_texts(labels)
        self.y_train = tokenizer.texts_to_sequences(labels)
        self.label_length = [len(label) for label in self.y_train]
        return tokenizer

    def create_subwordpiece_data(self,path, csv_path):
          
        train_csv = pd.read_csv(csv_path)
        for i in range(len(train_csv["text"])):
            if (i>=100):
                break
            filename = train_csv.iloc[i]["filename"]
            audio_vec = load_audio(os.path.join(path, filename))
            feature_vec = compute_mel_spec(audio_vec)
            self.x_train.append(feature_vec)
            label = train_csv.iloc[i]["text"]
            labels = bpemb_en.encode(label)
            labels = [bpemb_en.words.index(j) for j in label]
            self.y_train.append(labels)

    def steps_per_epoch(self, batch_size = 32):
        return len(self.x_train)//batch_size

    def next_batch(self, batch_size=32):

        current = 0

        while True:

            if (current + batch_size >= len(self.x_train)):
                current = 0

            self.x_train, self.logit_length, self.label_length, self.y_train= sklearn.utils.shuffle(self.x_train, self.logit_length, self.y_train, self.label_length)

            batch_y = self.y_train[current : current + batch_size]
            batch_logit_length = self.logit_length[current : current + batch_size]
            batch_label_length = self.label_length[current : current + batch_size]
            batch_x = self.x_train[current : current + batch_size]

            current += batch_size
            
            ret = []
            ret.append(batch_x)
            ret.append(batch_logit_length)
            ret.append(batch_y)
            ret.append(batch_label_length)
            yield ret

    def pad_feature(self, x):
      max_len = max(self.logit_length)
      pad_features = []

      for i in range(len(x)):
        len_lag = max_len - self.logit_length[i]
        arr_lag = np.zeros((len_lag, np.shape(x[i])[-1]))
        pad_feature = np.concatenate((x[i], arr_lag))
        pad_features.append(pad_feature)

      return pad_features

    def pad_label(self, y):
      max_len = max(self.label_length)
      pad_labels = []

      for i in range(len(y)):
        len_lag = max_len - self.label_length[i]
        arr_lag = np.zeros((len_lag), dtype=np.int64)
        pad_label = np.concatenate((y[i], arr_lag))
        pad_labels.append(pad_label)

      return pad_labels


**MODEL CTC**

In [ ]:
n_mel_bins = 80

def build_model(vocab_size, n_mel_bins=80, num_layers = 3):
    mel_specs = tf.keras.layers.Input(shape=[None, n_mel_bins])
    norm_mel_specs = tf.keras.layers.LayerNormalization(name='norm0')(mel_specs)

    output = norm_mel_specs

    for i in range(num_layers):
        output = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True, name = 'lstm{}'.format(i)))(output)
        output = tf.keras.layers.LayerNormalization(name='norm{}'.format(i+1))(output)

    output = tf.keras.layers.Dropout(0.3, name = 'dropout1')(output)
    output = tf.keras.layers.Dense(vocab_size, name = 'dense1')(output)

    model =  tf.keras.Model(inputs = mel_specs, outputs = output)

    return model

**MODEL RNNT**

In [ ]:
def conv_layers(shape):
    input = tf.keras.layers.Input(shape = shape)
    input = tf.expand_dims(input, axis = -1)
    conv1 = tf.keras.layers.Conv2D(1, (3, shape[-1]))(input)
    conv1 = tf.reshape(conv1, shape=tf.shape(conv1)[:-1])
    return conv1

def encoder(specs_shape, num_layers = 3):
    mel_specs = tf.keras.layers.Input(shape = specs_shape)
    norm_mel_specs = tf.keras.layers.BatchNormalization()(mel_specs)

    output = norm_mel_specs
    
    for i in range(num_layers):
        output = tf.keras.layers.LSTM(2048, return_sequences=True)(output)
        output = tf.keras.layers.Dropout(0.3)(output)
        output = tf.keras.layers.LayerNormalization()(output)

    return tf.keras.Model(inputs = mel_specs, outputs = output)

def prediction_network(vocab_size, embed_size, num_layers = 1, dropout = 0.2):
    inputs = tf.keras.layers.Input(shape= [None])

    embed = tf.keras.layers.Embedding(vocab_size, embed_size)(inputs)

    output = embed
    for _ in range(num_layers):
        output = tf.keras.layers.LSTM(2048, return_sequences = True)(output)
        output = tf.keras.layers.Dropout(dropout)(output)
        output = tf.keras.layers.LayerNormalization()(output)

    return tf.keras.Model(inputs = inputs, outputs = output)

def build_keras_model(vocab_size, n_mel_bins = 80, embed_size = 500, reduction_factor = 3):

    mel_specs = tf.keras.layers.Input(shape = [None, n_mel_bins])
    pred_inp = tf.keras.layers.Input(shape = [None])

    inp_enc = encoder(specs_shape = [None, n_mel_bins])(mel_specs)
    pred_outputs = prediction_network(vocab_size = vocab_size, embed_size = embed_size)(pred_inp)

    joint_layer = (tf.expand_dims(inp_enc, axis = 2) + tf.expand_dims(pred_outputs, axis = 1))

    outputs = tf.keras.layers.Dense(640, activation = 'tanh')(joint_layer)
    outputs = tf.keras.layers.Dense(vocab_size, activation = 'softmax')(outputs)

    return tf.keras.Model(inputs = [mel_specs, pred_inp], outputs = outputs)


**LOSS RNNT**

In [ ]:
def forward_pass(log_probs, labels):

    T, U, _ = np.shape(log_probs)
    alphas = np.zeros((T, U))

    for t in range(1, T):
        alphas[t][0] = alphas[t-1][0] + log_probs[t-1][0][0]

    for u in range(1, U):
        
        alphas[0][u] = alphas[0][u-1] + log_probs[0][u-1][labels[u-1]]

    for t in range(1, T):
        for u in range(1, U):
            
            no_emit = alphas[t-1][u] + log_probs[t-1][u][0]
            emit = alphas[t][u-1] + log_probs[t][u-1][labels[u-1]]
            alphas[t][u] = np.logaddexp(no_emit, emit)

    loglike = alphas[T-1][U-1] + log_probs[T-1][U-1][0]

    return -loglike

def loss_rnnt(log_probs, labels):

    costs = []

    for b in range(len(log_probs)):
        #t = f_len[b]
        #u = g_len[b] + 1
        loss = forward_pass(log_probs[b], labels[b])
        
        costs.append(loss)

    costs = tf.reduce_mean(costs)
    return costs


**TRAINING**

In [ ]:
#vocab = init_vocab()

In [ ]:

path = "common-voice/cv-valid-train"
csv_path = "common-voice/cv-valid-train.csv"

train = DataGen()
tokenizer = train.create_wordpiece_data(path, csv_path)

In [ ]:
batch_size = 5

optimizer = tf.keras.optimizers.Adam()
steps_per_epoch = train.steps_per_epoch(batch_size)
generator = train.next_batch(batch_size)

In [ ]:
#model_ctc = build_model(len(vocab))

In [ ]:
vocab_len = len(tokenizer.word_index.items()) + 1 # for wordpiece model

In [ ]:
model_rnnt = build_keras_model(vocab_len)

In [ ]:
def run_train(n_epochs,model, steps_per_epoch, model_type = 'ctc'):

    if (model_type == 'ctc'):

        model_name = 'model_ctc'

        model = build_model(len(vocab))

        for epoch in range(n_epochs):
            for step in range(steps_per_epoch):

                data = next(generator)
                features, logit_length, labels, label_length = data[0], data[1], data[2], data[3]

                with tf.GradientTape() as tape:
                    logits = model(features)
                    loss = tf.nn.ctc_loss(labels, logits, label_length, logit_length, logits_time_major=False)
                    loss = tf.reduce_mean(loss)

                gradients = tape.gradient(loss, model.variables)

                optimizer.apply_gradient(zip(gradients, model.variables))

                if step % 25 == 0 and step != 0 :
                    print("Epoch {} , Step {} , Loss {}".format(epoch, step, loss))

            if epoch % 10 == 0 and epoch != 0:
                model.save_weights('weights/{}_epoch_{}'.format(model_name, epoch))

    elif (model_type == 'rnnt'):
        
        model_name = 'model_rnnt'

        for epoch in range(n_epochs):
            for step in range(steps_per_epoch):
                start = time.time()
                data = next(generator)
                #features, f_len, labels, g_len = np.array(data[0]), data[1], np.array(data[2]), data[3]
                features, labels = data[0], data[2]
                
                with tf.GradientTape() as tape:
                    y_pred = []
                    for b in range(len(labels)):
                      feature = np.expand_dims(features[b], axis = 0)
                      label = [0] + labels[b]
                      pred_input = np.expand_dims(label, axis = 0)
                      probs = model([feature, pred_input])[0]
                      log_probs = tf.math.log(probs)
                      y_pred.append(log_probs)
                    
                    loss = loss_rnnt(y_pred, labels)
                gradients = tape.gradient(loss, model.variables)
                optimizer.apply_gradients(zip(gradients, model.variables))
                print(time.time() - start)
                print("Epoch {} , Step {} , Loss {}".format(epoch, step, loss))
            
            if epoch % 10 == 0 and epoch != 0:
                model.save_weights('weights/{}_epoch_{}'.format(model_name, epoch))


**CTC TEST**

In [ ]:
def evaluate(x_test):

    vocab = init_vocab()
    model = build_model(len(vocab))

    model.load_weights('weights/model_ctc_30')

    dim = len(np.shape(x_test))
    if (dim == 2):
        mel_feature = compute_mel_spec(x_test)
        x = np.expand_dims(mel_feature, axis=0)
    elif (dim == 3):
        x = []
        for i in x_test:
            mel_feature = compute_mel_spec(i)
            x.append(mel_feature)

    y_pred = model(x)
    decoded_y_pred = ctc_greedy_decode(y_pred)

    return decoded_y_pred

**RNNT TEST**

In [ ]:
#Tokenize/Wordpiece test
reverse_word_map = dict (map(reversed, tokenizer.word_index.items()))

In [ ]:
def evaluate(x_test, reverse_word_map, wordpiece = True):
    vocab = init_vocab()
    model = build_keras_model(len(vocab))

    model.load_weights('weights_path')
    y_pred = []
    
    for i in x_test:
        pred_text = ""
        mel_feature = compute_mel_spec(i)
        y_decoded = rnnt_greedy_decode(model, mel_feature)
        if wordpiece:
          y_decoded = [reverse_word_map[index] for index in y_decoded]
        for index in y_decoded:
            pred_text = pred_text + " "
        y_pred.append(pred_text[:-1])

    y_pred = rnnt_greedy_decode(model, x)
    
    return y_pred

In [ ]:
def eval_wer(path, tokenizer, csv_path, reverse_word_map):
  test_csv = pd.read_csv(csv_path)
        labels = []
        x_test = []
        for i in range(len(test_csv["text"])):
            if (i>=100):
                break
            filename = test_csv.iloc[i]["filename"]
            audio_vec = load_audio(os.path.join(path, filename))
            x_test.append(audio_vec)
            label = test_csv.iloc[i]["text"]
            labels.append(label)

        y_pred = evaluate(x_test, reverse_word_map)
        return wer(labels, y_pred)
